# Investigation 2: Fix Defensive Stats Bug

**Problem**: Defensive stats showed -3.9% impact (worst individual feature)

**Suspected Bug**:
```python
features['defensive_ppg'] = defensive_plays['epa'].sum() * 6 / weeks
```

**Why multiply by 6?** This artificially inflates defensive points allowed by 6x!

**Expected**: Fixing this should flip defensive stats from -3.9% to +1-2% (helpful)

**Approach**:
1. Copy ablation study code
2. Fix defensive formula (remove × 6)
3. Re-run test #6 (Baseline + Defensive Stats)
4. Compare results

## Setup

In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, brier_score_loss, roc_auc_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported")

✅ Libraries imported


## Load Data (Reuse from Ablation Study)

In [2]:
print("Loading NFL data...")
print("(This takes 5-10 minutes on first run)\n")

START_YEAR = 2015
END_YEAR = 2024
TEST_YEAR = 2024

pbp_data = nfl.import_pbp_data(years=range(START_YEAR, END_YEAR + 1))
print(f"✅ Loaded {len(pbp_data):,} plays")

weekly_data = nfl.import_weekly_data(years=range(START_YEAR, END_YEAR + 1))
print(f"✅ Loaded {len(weekly_data):,} player-game records")

schedule_data = nfl.import_schedules(years=range(START_YEAR, END_YEAR + 1))
print(f"✅ Loaded {len(schedule_data):,} scheduled games")

Loading NFL data...
(This takes 5-10 minutes on first run)

2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.
✅ Loaded 483,605 plays
Downcasting floats.
✅ Loaded 54,479 player-game records
✅ Loaded 2,743 scheduled games


## Define Feature Engineering Functions

### ORIGINAL (Buggy) Defensive Stats

In [3]:
def add_defensive_features_BUGGY(features, pbp_data, team, season, week):
    """
    BUGGY VERSION: Multiplies EPA by 6 (artificially inflates defensive points)
    """
    defensive_plays = pbp_data[
        (pbp_data['defteam'] == team) &
        (pbp_data['season'] == season) &
        (pbp_data['week'] < week)
    ]
    
    if len(defensive_plays) > 0:
        weeks = defensive_plays['week'].nunique()
        features['defensive_ypg'] = defensive_plays['yards_gained'].sum() / weeks if weeks > 0 else 0
        # BUG: Why multiply by 6?
        features['defensive_ppg'] = defensive_plays['epa'].sum() * 6 / weeks if weeks > 0 else 0
    else:
        features['defensive_ypg'] = 0
        features['defensive_ppg'] = 0
    
    return features

print("✅ Buggy defensive function defined (for comparison)")

✅ Buggy defensive function defined (for comparison)


### FIXED Defensive Stats

In [4]:
def add_defensive_features_FIXED(features, pbp_data, team, season, week):
    """
    FIXED VERSION: Removed × 6 multiplication
    """
    defensive_plays = pbp_data[
        (pbp_data['defteam'] == team) &
        (pbp_data['season'] == season) &
        (pbp_data['week'] < week)
    ]
    
    if len(defensive_plays) > 0:
        weeks = defensive_plays['week'].nunique()
        features['defensive_ypg'] = defensive_plays['yards_gained'].sum() / weeks if weeks > 0 else 0
        # FIXED: Removed × 6
        features['defensive_epa_pg'] = defensive_plays['epa'].sum() / weeks if weeks > 0 else 0
    else:
        features['defensive_ypg'] = 0
        features['defensive_epa_pg'] = 0
    
    return features

print("✅ Fixed defensive function defined")
print("\n🔍 KEY CHANGE: defensive_ppg × 6 → defensive_epa_pg (no multiplier)")

✅ Fixed defensive function defined

🔍 KEY CHANGE: defensive_ppg × 6 → defensive_epa_pg (no multiplier)


## Legacy Features (Baseline)

In [5]:
def create_team_features_legacy(weekly_data, team, season, week):
    """
    Legacy Week 9 feature engineering (baseline).
    """
    team_stats = weekly_data[
        (weekly_data['recent_team'] == team) &
        (weekly_data['season'] == season) &
        (weekly_data['week'] < week)
    ]
    
    if team_stats.empty:
        return {}
    
    features = {
        'passing_ypg': team_stats['passing_yards'].sum() / len(team_stats['week'].unique()) if not team_stats.empty else 0,
        'rushing_ypg': team_stats['rushing_yards'].sum() / len(team_stats['week'].unique()) if not team_stats.empty else 0,
        'total_ypg': (team_stats['passing_yards'].sum() + team_stats['rushing_yards'].sum()) / len(team_stats['week'].unique()) if not team_stats.empty else 0,
        'points_pg': team_stats.groupby('week')['fantasy_points'].sum().mean() if not team_stats.empty else 0,
        'passing_tds_pg': team_stats['passing_tds'].sum() / len(team_stats['week'].unique()) if not team_stats.empty else 0,
        'turnovers_pg': (team_stats['interceptions'].sum() + team_stats.get('fumbles_lost', pd.Series([0])).sum()) / len(team_stats['week'].unique()) if not team_stats.empty else 0,
    }
    
    return features

print("✅ Legacy feature function defined")

✅ Legacy feature function defined


## Simple Predictor Class (No Configuration Needed)

In [6]:
class DefensiveTestPredictor:
    """
    Simplified predictor for testing defensive stats fix.
    """
    
    def __init__(self, use_fixed_defensive=True):
        self.use_fixed = use_fixed_defensive
        self.model = None
    
    def build_features(self, pbp_data, weekly_data, schedule_data, season, week):
        games = schedule_data[
            (schedule_data['season'] == season) &
            (schedule_data['week'] == week)
        ]
        
        X_list = []
        y_list = []
        
        for _, game in games.iterrows():
            home_team = game['home_team']
            away_team = game['away_team']
            
            # Legacy features
            home_features = create_team_features_legacy(weekly_data, home_team, season, week)
            away_features = create_team_features_legacy(weekly_data, away_team, season, week)
            
            if not home_features or not away_features:
                continue
            
            # Add defensive features (fixed or buggy)
            if self.use_fixed:
                home_features = add_defensive_features_FIXED(home_features, pbp_data, home_team, season, week)
                away_features = add_defensive_features_FIXED(away_features, pbp_data, away_team, season, week)
            else:
                home_features = add_defensive_features_BUGGY(home_features, pbp_data, home_team, season, week)
                away_features = add_defensive_features_BUGGY(away_features, pbp_data, away_team, season, week)
            
            # Combine
            combined = {}
            for key in home_features.keys():
                combined[f'home_{key}'] = home_features[key]
                combined[f'away_{key}'] = away_features.get(key, 0)
            
            combined['scoring_advantage'] = home_features.get('points_pg', 0) - away_features.get('points_pg', 0)
            combined['turnover_advantage'] = away_features.get('turnovers_pg', 0) - home_features.get('turnovers_pg', 0)
            combined['is_playoff'] = 1 if week >= 18 else 0
            combined['season'] = season
            
            X_list.append(combined)
            
            if pd.notna(game.get('home_score')) and pd.notna(game.get('away_score')):
                y_list.append(1 if game['home_score'] > game['away_score'] else 0)
            else:
                y_list.append(None)
        
        X = pd.DataFrame(X_list)
        y = pd.Series(y_list)
        
        valid_mask = y.notna()
        return X[valid_mask], y[valid_mask]
    
    def train(self, X_train, y_train):
        # Simple 3-model ensemble, depth=5
        rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
        lr = LogisticRegression(C=1.0, max_iter=1000, random_state=42)
        xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=200, random_state=42)
        
        voting = VotingClassifier([('rf', rf), ('lr', lr), ('xgb', xgb_model)], voting='soft')
        self.model = CalibratedClassifierCV(voting, method='isotonic', cv=3)
        self.model.fit(X_train, y_train)
    
    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        y_proba = self.model.predict_proba(X_test)[:, 1]
        return y_pred, y_proba

print("✅ DefensiveTestPredictor class created")

✅ DefensiveTestPredictor class created


## Test Buggy vs Fixed on Sample Weeks

Quick test on a few weeks to see the difference:

In [7]:
print("Testing BUGGY vs FIXED defensive stats on Week 15-17 of 2024...\n")

for use_fixed in [False, True]:
    version = "FIXED" if use_fixed else "BUGGY"
    predictor = DefensiveTestPredictor(use_fixed_defensive=use_fixed)
    
    print(f"\n{'='*60}")
    print(f"Testing {version} version")
    print(f"{'='*60}")
    
    all_correct = 0
    all_total = 0
    
    for test_week in [15, 16, 17]:
        # Train on all prior data
        X_train_list = []
        y_train_list = []
        
        for year in range(2015, 2024):
            for week in range(1, 19):
                X, y = predictor.build_features(pbp_data, weekly_data, schedule_data, year, week)
                if len(X) > 0:
                    X_train_list.append(X)
                    y_train_list.append(y)
        
        for week in range(1, test_week):
            X, y = predictor.build_features(pbp_data, weekly_data, schedule_data, 2024, week)
            if len(X) > 0:
                X_train_list.append(X)
                y_train_list.append(y)
        
        X_train = pd.concat(X_train_list, ignore_index=True)
        y_train = pd.concat(y_train_list, ignore_index=True)
        
        # Test
        X_test, y_test = predictor.build_features(pbp_data, weekly_data, schedule_data, 2024, test_week)
        
        if len(X_test) > 0:
            predictor.train(X_train, y_train)
            y_pred, y_proba = predictor.predict(X_test)
            
            acc = accuracy_score(y_test, y_pred)
            print(f"  Week {test_week}: {acc:.1%} ({(y_pred == y_test).sum()}/{len(y_test)})")
            
            all_correct += (y_pred == y_test).sum()
            all_total += len(y_test)
    
    overall_acc = all_correct / all_total if all_total > 0 else 0
    print(f"\n  Overall (Weeks 15-17): {overall_acc:.1%} ({all_correct}/{all_total})")

Testing BUGGY vs FIXED defensive stats on Week 15-17 of 2024...


Testing BUGGY version
  Week 15: 81.2% (13/16)
  Week 16: 81.2% (13/16)
  Week 17: 68.8% (11/16)

  Overall (Weeks 15-17): 77.1% (37/48)

Testing FIXED version


KeyboardInterrupt: 

## Full 2024 Test: Rerun Ablation Study Test #6

Complete walk-forward validation on all of 2024:

In [ ]:
def run_full_test(use_fixed_defensive=True):
    """
    Replicate Test #6 from ablation study with fixed defensive stats.
    """
    version = "FIXED" if use_fixed_defensive else "BUGGY (Original)"
    
    print(f"\n{'='*70}")
    print(f"Full 2024 Test: Baseline + Defensive Stats ({version})")
    print(f"{'='*70}")
    
    predictor = DefensiveTestPredictor(use_fixed_defensive=use_fixed_defensive)
    
    all_preds = []
    all_actuals = []
    all_probas = []
    
    for test_week in range(1, 19):
        print(f"  Testing week {test_week}...", end=" ")
        
        X_train_list = []
        y_train_list = []
        
        # Historical data
        for train_year in range(2015, 2024):
            for train_week in range(1, 19):
                X_week, y_week = predictor.build_features(pbp_data, weekly_data, schedule_data, train_year, train_week)
                if len(X_week) > 0:
                    X_train_list.append(X_week)
                    y_train_list.append(y_week)
        
        # 2024 up to test_week-1
        for train_week in range(1, test_week):
            X_week, y_week = predictor.build_features(pbp_data, weekly_data, schedule_data, 2024, train_week)
            if len(X_week) > 0:
                X_train_list.append(X_week)
                y_train_list.append(y_week)
        
        if len(X_train_list) == 0:
            print("No training data")
            continue
        
        X_train = pd.concat(X_train_list, ignore_index=True)
        y_train = pd.concat(y_train_list, ignore_index=True)
        
        X_test, y_test = predictor.build_features(pbp_data, weekly_data, schedule_data, 2024, test_week)
        
        if len(X_test) == 0:
            print("No test data")
            continue
        
        predictor.train(X_train, y_train)
        y_pred, y_proba = predictor.predict(X_test)
        
        all_preds.extend(y_pred)
        all_actuals.extend(y_test)
        all_probas.extend(y_proba)
        
        week_acc = accuracy_score(y_test, y_pred)
        print(f"Acc: {week_acc:.1%}")
    
    # Calculate metrics
    accuracy = accuracy_score(all_actuals, all_preds)
    brier = brier_score_loss(all_actuals, all_probas)
    auc = roc_auc_score(all_actuals, all_probas)
    
    # High-confidence
    hc_mask = np.array([(p > 0.70 or p < 0.30) for p in all_probas])
    if hc_mask.sum() > 0:
        hc_preds = np.array(all_preds)[hc_mask]
        hc_actuals = np.array(all_actuals)[hc_mask]
        hc_accuracy = accuracy_score(hc_actuals, hc_preds)
    else:
        hc_accuracy = None
    
    print(f"\n📊 RESULTS ({version}):")
    print(f"  Accuracy: {accuracy:.1%}")
    print(f"  HC Accuracy: {hc_accuracy:.1%}" if hc_accuracy else "  HC Accuracy: N/A")
    print(f"  Brier Score: {brier:.3f}")
    print(f"  AUC-ROC: {auc:.3f}")
    print(f"  Games: {len(all_actuals)}")
    
    return accuracy, hc_accuracy, brier

# Run both versions
print("\nRunning full 2024 test with BOTH versions...")
print("(This takes ~15-20 minutes)\n")

buggy_acc, buggy_hc, buggy_brier = run_full_test(use_fixed_defensive=False)
fixed_acc, fixed_hc, fixed_brier = run_full_test(use_fixed_defensive=True)

## Compare Results

In [ ]:
print("\n" + "="*70)
print("COMPARISON: BUGGY vs FIXED Defensive Stats")
print("="*70)

print(f"\n{'Metric':<20} {'BUGGY (× 6)':<20} {'FIXED':<20} {'Delta':<15}")
print("-"*70)

delta_acc = (fixed_acc - buggy_acc) * 100
delta_hc = (fixed_hc - buggy_hc) * 100 if buggy_hc and fixed_hc else 0
delta_brier = fixed_brier - buggy_brier

print(f"{'Accuracy':<20} {buggy_acc:>18.1%} {fixed_acc:>18.1%} {delta_acc:>+13.1f} pts")
if buggy_hc and fixed_hc:
    print(f"{'HC Accuracy':<20} {buggy_hc:>18.1%} {fixed_hc:>18.1%} {delta_hc:>+13.1f} pts")
print(f"{'Brier Score':<20} {buggy_brier:>18.3f} {fixed_brier:>18.3f} {delta_brier:>+13.3f}")

print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)

if delta_acc > 0.5:
    print(f"\n✅ CONFIRMED: Bug fixed! Defensive stats improved by {delta_acc:.1f} percentage points")
    print(f"   This validates the hypothesis that × 6 was causing the -3.9% harm.")
elif delta_acc < -0.5:
    print(f"\n❌ UNEXPECTED: Fixed version performed WORSE by {abs(delta_acc):.1f} percentage points")
    print(f"   The × 6 may have been intentional or there's another issue.")
else:
    print(f"\n⚪ NEUTRAL: Minimal difference ({delta_acc:.1f} pts)")
    print(f"   The × 6 bug may not be the primary cause of defensive stats issues.")

print("\nVs Baseline (66.8% from ablation study):")
baseline = 0.668
buggy_vs_baseline = (buggy_acc - baseline) * 100
fixed_vs_baseline = (fixed_acc - baseline) * 100

print(f"  Buggy defensive: {buggy_vs_baseline:+.1f} pts vs baseline")
print(f"  Fixed defensive: {fixed_vs_baseline:+.1f} pts vs baseline")

if fixed_vs_baseline > 1.0:
    print(f"\n✅ CONCLUSION: Fixed defensive stats are HELPFUL (+{fixed_vs_baseline:.1f}%)")
    print(f"   Recommendation: KEEP defensive stats with fixed formula")
elif fixed_vs_baseline < -1.0:
    print(f"\n❌ CONCLUSION: Even fixed, defensive stats are HARMFUL ({fixed_vs_baseline:+.1f}%)")
    print(f"   Recommendation: REMOVE defensive stats entirely")
else:
    print(f"\n⚪ CONCLUSION: Defensive stats are NEUTRAL ({fixed_vs_baseline:+.1f}%)")
    print(f"   Recommendation: Skip defensive stats, not worth complexity")

## Save Results

In [ ]:
results_summary = {
    'buggy_accuracy': buggy_acc,
    'fixed_accuracy': fixed_acc,
    'delta': delta_acc / 100,
    'buggy_vs_baseline': buggy_vs_baseline / 100,
    'fixed_vs_baseline': fixed_vs_baseline / 100
}

pd.DataFrame([results_summary]).to_csv('investigation_2_results.csv', index=False)
print("\n✅ Results saved to: investigation_2_results.csv")

with open('investigation_2_findings.md', 'w') as f:
    f.write("# Investigation 2: Defensive Stats Bug Fix Results\n\n")
    f.write(f"**Date**: {pd.Timestamp.now().strftime('%Y-%m-%d')}\n\n")
    
    f.write("## Bug Description\n\n")
    f.write("```python\n")
    f.write("# BUGGY:\n")
    f.write("features['defensive_ppg'] = defensive_plays['epa'].sum() * 6 / weeks\n\n")
    f.write("# FIXED:\n")
    f.write("features['defensive_epa_pg'] = defensive_plays['epa'].sum() / weeks\n")
    f.write("```\n\n")
    
    f.write("## Results\n\n")
    f.write(f"- **Buggy version**: {buggy_acc:.1%} accuracy\n")
    f.write(f"- **Fixed version**: {fixed_acc:.1%} accuracy\n")
    f.write(f"- **Delta**: {delta_acc:+.1f} percentage points\n\n")
    
    f.write("## Recommendation\n\n")
    if fixed_vs_baseline > 1.0:
        f.write(f"✅ **KEEP** defensive stats with fixed formula ({fixed_vs_baseline:+.1f}% vs baseline)\n")
    elif fixed_vs_baseline < -1.0:
        f.write(f"❌ **REMOVE** defensive stats entirely ({fixed_vs_baseline:+.1f}% vs baseline)\n")
    else:
        f.write(f"⚪ **SKIP** defensive stats, not worth complexity ({fixed_vs_baseline:+.1f}% vs baseline)\n")

print("✅ Findings saved to: investigation_2_findings.md")
print("\n" + "="*70)
print("INVESTIGATION 2 COMPLETE")
print("="*70)